In [423]:
import numpy as np
import pandas as pd
import random, datetime
import locale # used to format currency
import re
from datetime import datetime
import os

<b style="color:blue;font-size:25px">
    1. Account Management:
</b>

In [424]:
def create_account(name,account_type,initial_balance,password):
    account_details={}
    account_number = generate_account_number()
    account_number = str(account_number) + name[:2] + account_type[:1]
    # print(account_number)
    initial_balance = locale.currency(initial_balance, grouping=True)
    accounts_details.append({
        "name":name,
        "account_type":account_type,
        "account_number":account_number,
        "mpin":password,
        "transactions": [(datetime.today(), "initial_balance", initial_balance)]
    })
    
    with open(f"{account_number}.txt", "x", encoding="utf-8") as f:
        welcome_note = (
            f"Welcome {name},\n"
            "State Bank of India.\n"
            f"Your {account_type} account has been created successfully.\n"
            f"mpin : {password}\n"
            f"Initial balance : {initial_balance}"
        )
        f.write(welcome_note)
    account_details = {
        "name":name,
        "account_type":account_type,
        "account_number":account_number,
        "initial_balance": initial_balance,
        "mpin":password
    }
    return account_details

In [425]:
def account_login(file_path, mpin):
    with open(file_path, "r", encoding="utf-8") as f:
        content = f.read()
    
    match = re.search(r'mpin\s*:\s*(\d{4})', content)
    
    if match:
        file_mpin = match.group(1)  # Extract the digits
        if str(mpin) == file_mpin:
            return {"status": True, "message": "Login successful"}
    
    return {"status": False, "message": "Login failed. Please check your MPIN."}

In [426]:
def view_transaction(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        content = f.read()
    transactions = re.findall(r'(Initial balance|Deposit|Withdrawal|Transfer)\s*:\s*₹\s*([\d,]+\.\d{2})', content)
    transactions_list=[]
    for t_type, amount_str in transactions:
        amount = float(amount_str.replace(",", ""))
        
        #if t_type == "Initial balance":
        transactions_list.append({
            "transaction_type":t_type,
            "amount":amount
        })
    transaction_details = pd.DataFrame(transactions_list)
    return transaction_details

In [427]:
def view_account(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()
        
        name_match = re.search(r"Welcome\s+(.+?),", text)
        name = name_match.group(1) if name_match else None
        
        account_match = re.search(r"Your\s+(.+?)\s+has been created", text)
        account_type = account_match.group(1) if account_match else None
        
        mpin_match = re.search(r"mpin\s*:\s*(\d{4})", text)
        mpin = mpin_match.group(1) if mpin_match else None
        
        df = pd.DataFrame([{
            "name": name,
            "account_type": account_type,
            "mpin": mpin
        }])

        return df

In [428]:
#customer_type = input("Please enter new to create account / old to access your account.")
def main_menu():
    
    while True:
        accounts_details = []
        locale.setlocale(locale.LC_ALL, 'en_IN.UTF-8')
        # account_number = ""
        # name = ""
        # account_type = ""
        
        # accounts["name"] = accounts["account_type"] = accounts["account_number"] = accounts["initial_balance"] = ""
        transactions_list = []
        print("\n===== BANK MENU =====")
        print("1. Open Account")
        print("2. View Account")
        print("3. Deposit")
        print("4. Withdraw")
        print("5. Transfer")
        print("6. View Transactions")
        print("7. Exit")
        customer_type = input("Enter choice: ")
        if customer_type=="1":
            name = input("Enter your name:")
            account_type = input("Type (savings/current):")
            password = input("Please enter MPIN:")
            initial_balance = float(input("Initial balance:"))
            accounts = create_account(name,account_type,initial_balance,password)
        
            # create_account return
            if not accounts:
                print("Oops!, Sorry for inconvenience.")
            else:
                print()
                print("Account created successfully.")
                for customer,detail in accounts.items():
                    print(f"{customer} : {detail}")
        elif customer_type=="2":
            account_number = str(input("Please enter your account number."))
            account_number = account_number+".txt"
            if os.path.exists(account_number):
                mpin = input("Please enter your mpin.")
                if len(mpin) == 4:
                    login = account_login(account_number,mpin)
                    if login['status'] == True:
                        view_account_details = view_account(account_number)
                        print(view_account_details.to_string(index=False))
                else:
                    print("Please enter correct mpin.")
            else:
                print("No account found.Please re-check account number.")
        elif customer_type=="3" or customer_type=="4" or customer_type=="5":
            account_number = str(input("Please enter your account number."))
            account_number = account_number+".txt"
            if os.path.exists(account_number):
                mpin = input("Please enter your mpin.")
                if len(mpin) == 4:
                    login = account_login(account_number,mpin)
                    if login['status'] == True:
                        transaction_complete = transaction_type(customer_type,account_number)
                        print(transaction_complete)
                else:
                    print("Please enter correct mpin.")
            else:
                print("No account found.Please re-check account number.")
        elif customer_type=="6":
            account_number = str(input("Please enter your account number."))
            account_number = account_number+".txt"
            if os.path.exists(account_number):
                mpin = input("Please enter your mpin.")
                if len(mpin) == 4:
                    login = account_login(account_number,mpin)
                    if login['status'] == True:
                        view_trans = view_transaction(account_number)
                        print(view_trans.to_string(index=False))
                else:
                    print("Please enter correct mpin.")
            else:
                print("No account found.Please re-check account number.")
        elif customer_type=="7":
            return print("Thank you for visiting.")

<b style="color:blue;font-size:25px">
    2. Transactions:
</b>

In [429]:
def calculate_balance(file_path,transaction_type):
    balance = 0.0

    with open(file_path, "r", encoding="utf-8") as f:
        content = f.read()

    # if transaction_type == "Deposit":
    #     transactions = re.findall(r'(Initial balance|Deposit)\s*:\s*₹\s*([\d,]+\.\d{2})', content)
    # elif transaction_type == "Withdrawal":
    #     transactions = re.findall(r'(Initial balance|Withdrawal)\s*:\s*₹\s*([\d,]+\.\d{2})', content)
    # elif transaction_type == "Transfer":
    #     transactions = re.findall(r'(Initial balance|Deposit|Withdrawal|Transfer)\s*:\s*₹\s*([\d,]+\.\d{2})', content)

    transactions = re.findall(r'(Initial balance|Deposit|Withdrawal|Transfer)\s*:\s*₹\s*([\d,]+\.\d{2})', content)
    
    for t_type, amount_str in transactions:
        amount = float(amount_str.replace(",", ""))

        if t_type == "Initial balance":
            balance = amount  # start with initial balance
        elif t_type == "Deposit":
            balance += amount
        elif t_type in ("Withdrawal", "Transfer"):
            balance -= amount

    return balance

In [430]:
def transaction_action(transaction_type,account_number):
    balance={}
    if transaction_type == "3":
        deposit_amount = int(input("Please enter amount to deposit."))
        if deposit_amount <=0:
            balance = {
                "status":False,"messgae":"Please enter positive number"
            }
            return balance
        else:
            deposit_amount = locale.currency(deposit_amount, grouping=True)
            #print(f"{deposit_amount} to be added in {accounts["account_number"]}.")
    
            today = datetime.today()
    
            entry_date = today.strftime("%d-%m-%Y")
            
            with open(f"{account_number}", "a", encoding="utf-8") as f:
              f.write(f"\nDeposit : {deposit_amount},Date: {entry_date}")
            
            file_path = account_number
            final_balance = calculate_balance(file_path,"Deposit")
            final_balance = f"Final Balance: ₹ {final_balance:,.2f}"
            balance = {
                "status":True,"messgae":"Deposit transaction completed.","final_balance":final_balance
            }
            return balance
        
    elif transaction_type == "4":

        withdrawal_amount = int(input("Please enter amount to withdrew."))
        file_path = account_number
        final_balance = calculate_balance(file_path,"Withdrawal")
        if withdrawal_amount <=0 or withdrawal_amount > final_balance or final_balance < 0:
            if withdrawal_amount <=0:
                balance = {
                    "status":False,"messgae":"Please enter positive number"
                }
                return balance
            elif withdrawal_amount > final_balance:
                balance = {
                    "status":False,"messgae":f"Withdrew amount is greater than balance.Please enter amount below or equal to ₹ {final_balance:,.2f}"
                }
                return balance
            elif final_balance < 0:
                balance = {
                    "status":False,"messgae":"Your balance is low to proceed the request."
                }
                return balance
        else:
            withdrawal_amount = locale.currency(withdrawal_amount, grouping=True)
            #print(f"{deposit_amount} to be added in {accounts["account_number"]}.")
    
            today = datetime.today()
    
            entry_date = today.strftime("%d-%m-%Y")
            
            with open(f"{account_number}", "a", encoding="utf-8") as f:
              f.write(f"\nWithdrawal : {withdrawal_amount},Date: {entry_date}")
            
            file_path = account_number
            final_balance = calculate_balance(file_path,"Withdrawal")
            final_balance = f"Final Balance: ₹ {final_balance:,.2f}"
            balance = {
                "status":True,"messgae":"Withdrew transaction completed.","final_balance":final_balance
            }
            return balance
            
    elif transaction_type == "5":

        transfer_amount = int(input("Please enter amount to withdrew."))
        file_path = account_number
        final_balance = calculate_balance(file_path,"Withdrawal")
        if transfer_amount <=0 or transfer_amount > final_balance or final_balance < 0:
            if transfer_amount <=0:
                balance = {
                    "status":False,"messgae":"Please enter positive number"
                }
                return balance
            elif transfer_amount > final_balance:
                balance = {
                    "status":False,"messgae":f"Transfer amount is greater than balance.Please enter amount below or equal to ₹ {final_balance:,.2f}"
                }
                return balance
            elif final_balance < 0:
                balance = {
                    "status":False,"messgae":"Your balance is low to proceed the request."
                }
                return balance
        else:
            transfer_amount = locale.currency(transfer_amount, grouping=True)
            #print(f"{deposit_amount} to be added in {accounts["account_number"]}.")
    
            today = datetime.today()
    
            entry_date = today.strftime("%d-%m-%Y")
            
            with open(f"{account_number}", "a", encoding="utf-8") as f:
              f.write(f"\nTransfer : {transfer_amount},Date: {entry_date}")
            
            file_path = account_number
            final_balance = calculate_balance(file_path,"Withdrawal")
            final_balance = f"Final Balance: ₹ {final_balance:,.2f}"
            balance = {
                "status":True,"messgae":"Transfer transaction completed.","final_balance":final_balance
            }
            return balance        

In [431]:
# print(accounts["name"])
# print(accounts["account_type"])
# print(accounts["account_number"])
# print(accounts["initial_balance"])
def transaction_type(choice,account_number):
        transaction_type = choice
    
        if transaction_type == "3" or transaction_type == "4" or transaction_type == "5":
            print("A/c number : ",account_number)
            transaction_status = transaction_action(transaction_type,account_number)
            if transaction_status['status'] == True:
                print(transaction_status['messgae'])
                print(transaction_status['final_balance'])
            else:
                print(transaction_status['messgae'])
        else:
            print("Invalid choice! Please enter 3, 4, or 5.")    

In [432]:
main_menu()


===== BANK MENU =====
1. Open Account
2. View Account
3. Deposit
4. Withdraw
5. Transfer
6. View Transactions
7. Exit


Enter choice:  1
Enter your name: Farook mohammad
Type (savings/current): savings
Please enter MPIN: 1234
Initial balance: 5000



Account created successfully.
name : Farook mohammad
account_type : savings
account_number : 8718034163962776Fas
initial_balance : ₹ 5,000.00
mpin : 1234

===== BANK MENU =====
1. Open Account
2. View Account
3. Deposit
4. Withdraw
5. Transfer
6. View Transactions
7. Exit


Enter choice:  2
Please enter your account number. 8718034163962776Fas
Please enter your mpin. 1234


           name    account_type mpin
Farook mohammad savings account 1234

===== BANK MENU =====
1. Open Account
2. View Account
3. Deposit
4. Withdraw
5. Transfer
6. View Transactions
7. Exit


...

Transfer transaction completed.
Final Balance: ₹ 5,800.00
None

===== BANK MENU =====
1. Open Account
2. View Account
3. Deposit
4. Withdraw
5. Transfer
6. View Transactions
7. Exit


Enter choice:  6
Please enter your account number. 8718034163962776Fas
Please enter your mpin. 1234


transaction_type  amount
 Initial balance  5000.0
         Deposit  4000.0
      Withdrawal  2000.0
        Transfer  1200.0

===== BANK MENU =====
1. Open Account
2. View Account
3. Deposit
4. Withdraw
5. Transfer
6. View Transactions
7. Exit


Enter choice:  7


Thank you for visiting.
